In [1]:
from paramiko import SSHClient, AutoAddPolicy
import subprocess
import os

In [135]:
class BackendManager:
    def __init__(self, server, uname):
        self.server = server
        self.uname = uname

        self.client = SSHClient()
        self.client.set_missing_host_key_policy(AutoAddPolicy())
        self.connected = False

    def connect(self):
        self.client.connect(self.server, username=self.uname)
        self.connected = True

    def close(self):
        self.client.close()
        self.connected = False

    def check_if_file_exists(self, file_path):
        if self.connected:
            cmd = "test -f {};".format(file_path)
            _, stdout, _ = self.client.exec_command(cmd)
            ret = stdout.channel.recv_exit_status()
            if stdout.channel.recv_exit_status() == 0:
                return True
            return False
        return -1


    def copy_from_backend(self, backend_path, local_path):
        call = 'scp {uname}@{server}:{backend_path} {local_path}'.format(uname=self.uname,
                                                                         server=self.server,
                                                                         backend_path=backend_path,
                                                                         local_path=local_path)
        print(call)
        try:
            ret = subprocess.check_output(call.split())
            print(ret)
            return 0
        except Exception as e:
            print(e)
            return 1


    def run_cmd(self, cmd):
        if self.connected:
            print(cmd)
            _, stdout, _ = self.client.exec_command(cmd)

            if stdout.channel.recv_exit_status() == 0:
                return stdout, 0
            print("Error: ", stdout.channel.recv_exit_status())
            return stdout, 1

        return None, -1

    def check_slrum_status(self, jobname):
        if self.connected:
            cmd = "squeue --format=\"%.18i %.9P %.30j %.8u %.8T %.10M %.9l %.6D %R\" --me"
            _, stdout, _ = self.client.exec_command(cmd)
            ret = stdout.readlines()
            for j in ret:
                if jobname in j.split():
                    print(j)
                    return 2

            print(ret)
            return 0
        return -1
        
    def debug_cmd(self, cmd):
        call = 'ssh -l {} {}'.format(self.uname, self.server).split()
        ret = subprocess.check_output(call + [cmd, ])
        print(ret)


In [168]:
bm = BackendManager("login18-1.hpc.itc.rwth-aachen.de", "as641651")

In [169]:
bm.connect()

In [138]:
bm.check_if_file_exists("pascal_voc.py")

True

In [139]:
bm.copy_from_backend("~/pascal_vo.py", ".")

scp as641651@login18-1.hpc.itc.rwth-aachen.de:~/pascal_vo.py .
Command '['scp', 'as641651@login18-1.hpc.itc.rwth-aachen.de:~/pascal_vo.py', '.']' returned non-zero exit status 1.


scp: /home/as641651/pascal_vo.py: No such file or directory


1

In [140]:
bm.check_slrum_status("jj")

['             JOBID PARTITION                           NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)\n']


0

In [152]:
class Commands:
    def __init__(self, source=None):
        self.source = source
        
    def wrap_cmd_with_source(self,cmd):
        """Source a bash or zsh script before executing every command"""
        if self.source:
            source = "source {}; ".format(self.source)
            return source+cmd
        return cmd
    
    def build_cmd(self, app, script_path, args=""):
        """For execution on interactive nodes"""
        script_dir, script = os.path.split(script_path)
        cmd= ""
        if script_dir  != '':
            cmd += "cd {}; ".format(script_dir)
        cmd += "{} {} {}".format(app, script, args)
        return self.wrap_cmd_with_source(cmd)
    
    def submit_job_cmd(self, submit_cmd, app, script_path, args=""):
        """For batch submissions"""
        script_dir, script = os.path.split(script_path)
        cmd= ""
        if script_dir  != '':
            cmd += "cd {}; ".format(script_dir)
        cmd += "{} {} '{} {}'".format(submit_cmd, app, script, args)
        return self.wrap_cmd_with_source(cmd)
        

In [142]:
cmd = Commands(source="~/.analyzer")

In [143]:
cmd.wrap_cmd_with_source("cd")

'source ~/.analyzer; cd'

In [144]:
cmd.build_cmd("python", "~/PHD/main.py", "0")

'source ~/.analyzer; cd ~/PHD; python main.py 0'

In [145]:
class MyCommands(Commands):
    def __init__(self, source="", submit_cmd=None):
        super().__init__(source)
        
        self.submit_cmd = submit_cmd
        
        self.interactive = True
        if self.submit_cmd:
            self.interactive = False
        
    def run_julia_cmd(self, script_path, args=""):
        if  self.interactive:
            cmd = self.build_cmd("julia", script_path, args)
        else:
            cmd = self.submit_job_cmd(self.submit_cmd, "julia", script_path, args)
        return cmd
    
    def run_python_cmd(self, script_path, args=""):
        if  self.interactive:
            cmd = self.build_cmd("python", script_path, args)
        else:
            cmd = self.submit_job_cmd(self.submit_cmd, "python", script_path, args)
        return cmd

In [153]:
mc= MyCommands()

In [154]:
mc.build_cmd("python", "~/PHD/main.py", "0")

'cd ~/PHD; python main.py 0'

In [155]:
mc.run_julia_cmd("~/PhD/runner.jl")

'cd ~/PhD; julia runner.jl '

In [156]:
mc= MyCommands(source="~/.analyzer", submit_cmd="sbatch submit.sh")

In [157]:
mc.run_julia_cmd("~/PhD/runner.jl")

"source ~/.analyzer; cd ~/PhD; sbatch submit.sh julia 'runner.jl '"

In [172]:
j_file = "~/PhD/performance-analyazer/Experiment2/Matrix-Chain-4/variants-linnea/experiments/100_100_100_100_100/runner.jl"

In [173]:
mc= MyCommands(source="~/.analyzer")
cmd = mc.run_julia_cmd(j_file)
cmd

'source ~/.analyzer; cd ~/PhD/performance-analyazer/Experiment2/Matrix-Chain-4/variants-linnea/experiments/100_100_100_100_100; julia runner.jl '

In [175]:
out, ret_code = bm.run_cmd(cmd)

source ~/.analyzer; cd ~/PhD/performance-analyazer/Experiment2/Matrix-Chain-4/variants-linnea/experiments/100_100_100_100_100; julia runner.jl 


In [176]:
out.readlines()

[]